In [1]:
import pandas as pd
import numpy as np
from os.path import exists
from tqdm import tqdm
import matplotlib.pyplot as plt
import datatable as dt
from collections import defaultdict

# Variables that contains the file location
from files import *
from functions import *

In [2]:
# if we modify the file we need to reload it with this
import importlib
import functions #import the module here, so that it can be reloaded.
importlib.reload(functions)

<module 'functions' from 'C:\\Users\\Richi\\Google Drive\\Study\\5th Semester\\Multimedia Search and Retrieval\\Task 1\\Git\\MMSR\\project\\functions.py'>

# Data
Load the Data

## Song informations

In [3]:
# genres
genres  = dt.fread(file_genres_2).to_pandas()
genres.set_index('id', inplace=True)
# song infos (artist, song, album name)
info  = dt.fread(file_info_2).to_pandas()
info.set_index('id', inplace=True)

In [4]:
# convert genres to a list
#genres['genre']= genres.genre.apply(lambda x: get_genres(x))

In [5]:
genres.head(1)

,genre
id,
0009fFIM1eYThaPg,['pop']


In [6]:
info.head(1)

,artist,song,album_name
id,,,
0009fFIM1eYThaPg,Cheryl,Rain on Me,3 Words


## Lyrics based features

In [7]:
# lyrics based feature vectors
tf_idf  = dt.fread(file_tfidf_2)
tf_idf[dt.float64] = dt.float32
tf_idf = tf_idf.to_pandas()
tf_idf.set_index('id', inplace=True)

word2vec  = dt.fread(file_word2vec_2, header=True)
word2vec[dt.float64] = dt.float32
word2vec = word2vec.to_pandas()
word2vec.set_index('id', inplace=True)

bert = dt.fread(file_bert_2,header=True)
bert[dt.float64] = dt.float32
bert = bert.to_pandas()
bert.set_index('id', inplace=True)

## Audio features

### [Low-level] Essentia

In [8]:
essentia  = dt.fread(file_essentia)
essentia[dt.float64] = dt.float32
essentia = essentia.to_pandas()
essentia.set_index('id', inplace=True)
essentia.head(2)

,lowlevel.average_loudness,lowlevel.barkbands.mean_0,lowlevel.barkbands.mean_1,lowlevel.barkbands.mean_2,lowlevel.barkbands.mean_3,lowlevel.barkbands.mean_4,lowlevel.barkbands.mean_5,lowlevel.barkbands.mean_6,lowlevel.barkbands.mean_7,lowlevel.barkbands.mean_8,...,tonal.thpcp_30,tonal.thpcp_31,tonal.thpcp_32,tonal.thpcp_33,tonal.thpcp_34,tonal.thpcp_35,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio
id,,,,,,,,,,,,,,,,,,,,,
0009fFIM1eYThaPg,0.933530,0.003060,0.015665,0.004106,0.001265,0.001988,0.001152,0.000906,0.000356,0.001064,...,0.752843,0.544831,0.394721,0.372330,0.422420,0.855395,0.563373,0.226941,434.193115,0.944264
0010xmHR6UICBOYT,0.985564,0.009209,0.078349,0.004850,0.001458,0.000622,0.000331,0.001254,0.003589,0.000960,...,0.119888,0.325183,0.882466,0.917528,0.421067,0.563627,0.500238,0.263457,434.193115,0.981752


### [Mid-level] BLF: Block-Level features

In [9]:
# BLF: Block-Level features

blf_correlation  = dt.fread(file_blf_correlation)
blf_correlation[dt.float64] = dt.float32
blf_correlation = blf_correlation.to_pandas()
blf_correlation.set_index('id', inplace=True)

blf_deltaspectral  = dt.fread(file_blf_deltaspectral)
blf_deltaspectral[dt.float64] = dt.float32
blf_deltaspectral = blf_deltaspectral.to_pandas()
blf_deltaspectral.set_index('id', inplace=True)

blf_spectral  = dt.fread(file_blf_spectral)
blf_spectral[dt.float64] = dt.float32
blf_spectral = blf_spectral.to_pandas()
blf_spectral.set_index('id', inplace=True)

blf_spectralcontrast  = dt.fread(file_blf_spectralcontrast)
blf_spectralcontrast[dt.float64] = dt.float32
blf_spectralcontrast = blf_spectralcontrast.to_pandas()
blf_spectralcontrast.set_index('id', inplace=True)

blf_vardeltaspectral  = dt.fread(file_blf_vardeltaspectral)
blf_vardeltaspectral[dt.float64] = dt.float32
blf_vardeltaspectral = blf_vardeltaspectral.to_pandas()
blf_vardeltaspectral.set_index('id', inplace=True)

blf_logfluc  = dt.fread(file_blf_logfluc)
blf_logfluc[dt.float64] = dt.float32

# This is done because in the csv it has an extra column name, 
# so in case someone with the original dataset tries to run it, it fixes that error
# It looks weird, but it is because first i am loading the data into datatable and then pass it to dataframe
new_cols = ['id']
new_cols.extend(list(blf_logfluc.names[2:]))
new_cols = tuple(new_cols)
del blf_logfluc[:, -1]

blf_logfluc.names = new_cols
blf_logfluc = blf_logfluc.to_pandas()
blf_logfluc.set_index('id', inplace=True)

### [Mid-level] MFCC: Mel Frequency Cepstral Coefficients

In [10]:
mfcc_bow  = dt.fread(file_mfcc_bow)
mfcc_bow[dt.float64] = dt.float32
mfcc_bow = mfcc_bow.to_pandas()
mfcc_bow.set_index('id', inplace=True)

mfcc_stats  = dt.fread(file_mfcc_stats)
mfcc_stats[dt.float64] = dt.float32
mfcc_stats = mfcc_stats.to_pandas()
mfcc_stats.set_index('id', inplace=True)

## Video features

In [11]:
incp  = dt.fread(file_incp)
incp[dt.float64] = dt.float32
incp = incp.to_pandas()
incp.set_index('id', inplace=True)

resnet  = dt.fread(file_resnet)
resnet[dt.float64] = dt.float32
resnet = resnet.to_pandas()
resnet.set_index('id', inplace=True)

vgg19  = dt.fread(file_vgg19)
vgg19[dt.float64] = dt.float32
vgg19 = vgg19.to_pandas()
vgg19.set_index('id', inplace=True)

# Early Fusion

In [12]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA , KernelPCA, FastICA
from sklearn.feature_selection import SelectKBest, f_classif, chi2, SelectFpr
import functools as ft
import itertools

In [13]:
"""
# since to compute every set for the full dataset is to big we will take only a sample set to compute scores and check which set will be best.
# if we find a good set we can do it on the full dataset later. This Strategy should save some computation time.
# lets take 20%
sample_ids = info.sample(frac=0.2, random_state=101)
sample_ids
"""

'\n# since to compute every set for the full dataset is to big we will take only a sample set to compute scores and check which set will be best.\n# if we find a good set we can do it on the full dataset later. This Strategy should save some computation time.\n# lets take 20%\nsample_ids = info.sample(frac=0.2, random_state=101)\nsample_ids\n'

In [14]:
"""
# now we have sample ids and can take a sample from each dataframe. We will overwrite the original Dataset to save Memory.
tf_idf = tf_idf.loc[sample_ids.index.values]
word2vec = word2vec.loc[sample_ids.index.values]
bert = bert.loc[sample_ids.index.values]
essentia = essentia.loc[sample_ids.index.values]
blf_correlation = blf_correlation.loc[sample_ids.index.values]
blf_deltaspectral = blf_deltaspectral.loc[sample_ids.index.values]
blf_spectral = blf_spectral.loc[sample_ids.index.values]
blf_spectralcontrast = blf_spectralcontrast.loc[sample_ids.index.values]
blf_vardeltaspectral = blf_vardeltaspectral.loc[sample_ids.index.values]
blf_logfluc = blf_logfluc.loc[sample_ids.index.values]
mfcc_bow = mfcc_bow.loc[sample_ids.index.values]
mfcc_stats = mfcc_stats.loc[sample_ids.index.values]
incp = incp.loc[sample_ids.index.values]
resnet = resnet.loc[sample_ids.index.values]
vgg19 = vgg19.loc[sample_ids.index.values]


# just a check so each Dataframe have correct IDS
assert(np.all(sample_ids.index.values == tf_idf.index.values))
assert(np.all(sample_ids.index.values == word2vec.index.values))
assert(np.all(sample_ids.index.values == bert.index.values))
assert(np.all(sample_ids.index.values == essentia.index.values))
assert(np.all(sample_ids.index.values == blf_correlation.index.values))
assert(np.all(sample_ids.index.values == blf_deltaspectral.index.values))
assert(np.all(sample_ids.index.values == blf_spectral.index.values))
assert(np.all(sample_ids.index.values == blf_spectralcontrast.index.values))
assert(np.all(sample_ids.index.values == blf_vardeltaspectral.index.values))
assert(np.all(sample_ids.index.values == blf_logfluc.index.values))
assert(np.all(sample_ids.index.values == mfcc_bow.index.values))
assert(np.all(sample_ids.index.values == mfcc_stats.index.values))
assert(np.all(sample_ids.index.values == incp.index.values))
assert(np.all(sample_ids.index.values == resnet.index.values))
assert(np.all(sample_ids.index.values == vgg19.index.values))

"""

'\n# now we have sample ids and can take a sample from each dataframe. We will overwrite the original Dataset to save Memory.\ntf_idf = tf_idf.loc[sample_ids.index.values]\nword2vec = word2vec.loc[sample_ids.index.values]\nbert = bert.loc[sample_ids.index.values]\nessentia = essentia.loc[sample_ids.index.values]\nblf_correlation = blf_correlation.loc[sample_ids.index.values]\nblf_deltaspectral = blf_deltaspectral.loc[sample_ids.index.values]\nblf_spectral = blf_spectral.loc[sample_ids.index.values]\nblf_spectralcontrast = blf_spectralcontrast.loc[sample_ids.index.values]\nblf_vardeltaspectral = blf_vardeltaspectral.loc[sample_ids.index.values]\nblf_logfluc = blf_logfluc.loc[sample_ids.index.values]\nmfcc_bow = mfcc_bow.loc[sample_ids.index.values]\nmfcc_stats = mfcc_stats.loc[sample_ids.index.values]\nincp = incp.loc[sample_ids.index.values]\nresnet = resnet.loc[sample_ids.index.values]\nvgg19 = vgg19.loc[sample_ids.index.values]\n\n\n# just a check so each Dataframe have correct IDS\n

## Feature selection

In [15]:
# PCA feature selection, we can get only features so we still have 99% explained variance from the Data.
def PCA_selection(df_set):
    pca = PCA(n_components=0.99, svd_solver='auto')
    PCA_features = pd.DataFrame(pca.fit_transform(df_set))
    print (f' original shape: {df_set.shape}') 
    print (f' components = {pca.n_components_}, explained variance = {pca.explained_variance_ratio_.sum()}')
    print (f' features reduced by {(1-pca.n_components_/df_set.shape[1]) *100:.2f}%')
    return PCA_features

# We can use also PCA Kernel but we need to fix the number of components. Here we have different things to try
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘cosine’, ‘precomputed’}, default=’linear’
def PCA_Kernel_selection(df_set, kernel='rbf', components:int=100, gamma=None):
    kernel_pca = KernelPCA(n_components=components, kernel=kernel, gamma=gamma)
    kernel_PCA_features = pd.DataFrame(kernel_pca.fit_transform(df_set))
    return kernel_PCA_features

# Fast ICA would be another solution of dimensionality reduction
def ICA_selection(df_set, components:int=100):
    fast_ICA = FastICA(n_components=components)
    fast_ICA_features = pd.DataFrame(fast_ICA.fit_transform(df_set))
    return fast_ICA_features

# we could also try TSNE, but not sure about performance. maybe long computation time
def TSNE_selection(df_set, components:int=100):
    tsne = TSNE(n_components=components, learning_rate='auto', init='random')
    tsne_features = pd.DataFrame(tsne.fit_transform(mid_low_features))
    return tsne_features

In [16]:
# list of features for each category
text_features = {'tf_idf': tf_idf, 'word2vec':word2vec, 'bert':bert}
audio_features = {'essentia':essentia,
                  'blf_correlation':blf_correlation,
                  'blf_deltaspectral':blf_deltaspectral,
                  'blf_spectral':blf_spectral,
                  'blf_spectralcontrast':blf_spectralcontrast,
                  'blf_vardeltaspectral':blf_vardeltaspectral,
                  'blf_logfluc':blf_logfluc,
                  'mfcc_bow':mfcc_bow,
                  'mfcc_stats':mfcc_stats}
video_features = {'incp':incp, 'resnet':resnet, 'vgg19':vgg19}
all_features = {**text_features, **audio_features, **video_features}


# since if we want to try all different combinations this would be to much to handle. there are thousends of different combinations we need to concentrate on some.
# from previous task we know bert and resnet where the more important features. so lets take them and combine with different audio features.
def flatten(l):
    return [item for sublist in l for item in sublist]

set1 = flatten([['bert'], audio_features.keys(), ['resnet']]) # one set with 1 text, audio, 1 video
set2 = flatten([['bert'], audio_features.keys()]) # one set with 1 text, audio, without video
set3 = flatten([audio_features.keys(), ['resnet']]) # one set without text, audio, video
set4 = flatten([['bert'], ['resnet']]) # one set with text, video
set5 = flatten([['bert'], ['essentia'], ['resnet']])  # one set with text, essentia, video
set6 = flatten([['bert'], ['blf_correlation'], ['resnet']])
set7 = flatten([['bert'], ['blf_deltaspectral'], ['resnet']])
set8 = flatten([['bert'], ['essentia', 'blf_logfluc'], ['resnet']])
set9 = flatten([['bert'], ['essentia', 'blf_logfluc', 'mfcc_bow'], ['resnet']])
set10 = flatten([['bert'], ['essentia', 'blf_logfluc', 'mfcc_bow']])

allsets = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10]

# function for join sets, for later use. we are limited on RAM so do it only when we need the dataset.
def join_dfs(set_n, features):
    fset = [features[x] for x in set_n]
    return ft.reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), fset)

In [17]:
# already a combination of 5 features give 126 different combinations. To much to Test all!
result_list = list(itertools.combinations(audio_features.keys(), 5))
len(result_list)

126

In [18]:
# lets see how much we can reduce by PCA if we still want to have 99% explained variance:
pca_features={}
for name, val in zip(all_features.keys(), all_features.values()):
    print(name)
    pca_features[name] = PCA_selection(val)
    print('------------------')

tf_idf
 original shape: (68641, 1000)
 components = 957, explained variance = 0.9901756877480314
 features reduced by 4.30%
------------------
word2vec
 original shape: (68641, 300)
 components = 270, explained variance = 0.990314907049147
 features reduced by 10.00%
------------------
bert
 original shape: (68641, 768)
 components = 525, explained variance = 0.990061538059662
 features reduced by 31.64%
------------------
essentia
 original shape: (68641, 1034)
 components = 2, explained variance = 0.9918565334205097
 features reduced by 99.81%
------------------
blf_correlation
 original shape: (68641, 1326)
 components = 960, explained variance = 0.9900035135747746
 features reduced by 27.60%
------------------
blf_deltaspectral
 original shape: (68641, 1372)
 components = 362, explained variance = 0.9900050860046592
 features reduced by 73.62%
------------------
blf_spectral
 original shape: (68641, 980)
 components = 147, explained variance = 0.9901046629066191
 features reduced b

## Compute similarity and top IDs

In [19]:
def generate_ids(df, setname):
    # generate similarity
    result = compute_in_batches_distance(df.to_numpy(dtype=np.float32), df.to_numpy(dtype=np.float32), simfunction = get_cosine_similarity, batches=100)
    print(result.shape)
    np.fill_diagonal(result, -1)
    # save to file
    dt.Frame(pd.DataFrame(compute_in_batches_topIds(result,info.index.values,100,100), index=info.index.values).reset_index()).to_csv(f'./TopIdsTaskGenerated/top_ids_cosine_earlyfusion_{setname}_complete.csv')
    del result

In [20]:
# run for all sets
for i, s in enumerate(allsets):
    print(f'doing set{i+1}')
    joined_df = join_dfs(s, pca_features)
    generate_ids(joined_df, f'set{i+1}')

doing set1


V:\Temp\ipykernel_39452\2988892163.py:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x', '1_x'} in the result is deprecated and will raise a MergeError in a future version.
  return ft.reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), fset)
V:\Temp\ipykernel_39452\2988892163.py:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'14_x', '5_x', '46_x', '74_x', '49_x', '60_x', '41_x', '53_x', '136_x', '28_x', '17_x', '132_x', '107_x', '121_x', '76_x', '113_x', '111_x', '144_x', '16_x', '108_x', '95_x', '84_x', '19_x', '13_x', '117_x', '30_x', '88_x', '36_x', '42_x', '67_x', '112_x', '103_x', '92_x', '52_x', '124_x', '44_x', '109_x', '138_x', '10_x', '78_x', '105_x', '21_x', '143_x', '45_x', '97_x', '122_x', '86_x', '8_x', '6_x', '61_x', '68_x', '38_x', '118_x', '18_x', '58_x', '73_x', '100_x', '51_x', '129_x', '26_x', '77_x', '102_x', '9_x', '65_x', '47_x', '22_x', '99_x', '85_x', '66_x', '72_x', '116_x', '24_x'

(68641, 68641)


100%|██████████| 100/100 [08:28<00:00,  5.08s/it]


doing set2


...

100%|██████████| 100/100 [06:57<00:00,  4.17s/it]


doing set10


V:\Temp\ipykernel_39452\2988892163.py:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x', '1_x'} in the result is deprecated and will raise a MergeError in a future version.
  return ft.reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), fset)
100%|██████████| 100/100 [00:49<00:00,  2.00it/s]


(68641, 68641)


100%|██████████| 100/100 [07:08<00:00,  4.29s/it]


## Compute scores

In [22]:
print('AP, RR, ndcg')
for i in range(1,11):
    top_set = dt.fread(f'./TopIdsTaskGenerated/top_ids_cosine_earlyfusion_set{i}_complete.csv', header=True).to_pandas()
    top_set.set_index('index', inplace=True)
    print(f'Metrics set{i}:', getMetrics(top_set, 100, genres))
    print('-------------------')

AP, RR, ndcg


100%|██████████| 68641/68641 [02:51<00:00, 401.17it/s]


Metrics set1: (0.4713479340727472, 0.6048723570874691, 0.7041884452006223)
-------------------


100%|██████████| 68641/68641 [02:51<00:00, 401.14it/s]


Metrics set2: (0.4713474535656341, 0.604863519930501, 0.7041875188866166)
-------------------


...

Metrics set8: (0.4713642087708921, 0.6053001877535551, 0.7042024195186465)
-------------------


100%|██████████| 68641/68641 [02:48<00:00, 408.03it/s]


Metrics set9: (0.4713593727148014, 0.605282841332436, 0.7042003438747417)
-------------------


100%|██████████| 68641/68641 [02:48<00:00, 406.94it/s]

Metrics set10: (0.47135848672574293, 0.6052765217305164, 0.70419757729533)
-------------------


# Normalize Data

In [ ]:
from sklearn import preprocessing

standard_scaler = preprocessing.StandardScaler()
minmax_scaler = preprocessing.MinMaxScaler()

def normalize_df(df, post_processing_scaler):
    df_np_values = df.values  #returns a numpy array
    # scale
    df_normalized = post_processing_scaler.fit_transform(df_np_values)
    # reset index from original
    df_normalized = pd.DataFrame(df_normalized).set_index(df.index)
    df_normalized.columns = df.columns[:]
    return df_normalized